# PEC

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from pathlib import Path
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cellpy

ModuleNotFoundError: No module named 'cellpy'

In [2]:
conda install -n myenv ipython
conda activate myenv
python -m ipykernel install --user --name myenv --display-name "Python (myenv)"

'C:\\Users\\uleri\\Anaconda3\\python.exe'

In [ ]:
# C:\scripts\cellpy\dev_data\PEC
raw_file_path = Path('/scripts/cellpy/dev_data/PEC') / 'pec.csv'

In [ ]:
raw_file_path.is_file()

In [ ]:
pec = cellpy.get(instrument="pec_csv", filename=raw_file_path, cycle_mode="cathode", mass=50_000)

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
pec.get_cycle_numbers()

In [ ]:
steptable = pec.cell.steps
steptable.head(10)

In [ ]:
summary = pec.cell.summary
summary.head(5)

In [ ]:
c, v = pec.get_dcap(1)


In [ ]:
curve1 = hv.Scatter((c,v))
curve1

In [ ]:
from cellpy import log
log.setup_logging(default_level="DEBUG")

In [ ]:
from cellpy.utils import ica

In [ ]:
dv1, dq1 = ica.dqdv(
    v, c,
)

In [ ]:
converter = ica.Converter()
converter.pre_smoothing = False
converter.post_smoothing = False
converter.smoothing = False
converter.normalise = True
converter.voltage_fwhm = 0.01
converter.max_points = 400
converter.capacity_resolution = 5.0
converter.voltage_resolution = 0.01
converter.set_data(c, v)
converter.inspect_data()
converter.pre_process_data()
converter.increment_data()
converter.post_process_data()
curve00 = hv.Scatter((c, v), label="as is").opts(width=800, height=500)
curve01 = hv.Scatter((converter.capacity_preprocessed, converter.voltage_preprocessed), label="preprocessed").opts(width=800, height=500)
curve00 * curve01

In [ ]:
print(len(converter._voltage_processed))
print(len(converter._incremental_capacity))
curve1 = hv.Curve((dv1, dq1), label="as is").opts(width=800, height=500)
curve2 = hv.Curve((converter.voltage_processed, converter.incremental_capacity), label="new").opts(width=800, height=500)
curve3 = hv.Curve((converter._voltage_processed, converter._incremental_capacity), label="pre").opts(width=800, height=500)
curve4 = hv.Curve((converter.voltage_processed, converter._incremental_capacity), label="shifted").opts(width=800, height=500)
curve1 * curve2 * curve3 * curve4

In [ ]:

dv1, dq1 = ica.dqdv(
    v, c,
    voltage_fwhm=0.01,
)


dv2, dq2 = ica.dqdv(
    v, c,
    voltage_fwhm=0.01,
    max_points=200,
)

curve2 = hv.Curve((dv1, dq1), label="as is").opts(width=800, height=500)
scatter2 = hv.Curve((dv2, dq2), label="1000 points").opts(width=800, height=500)
curve2 * scatter2

In [ ]:
?ica.dqdv